# Основы программирования в Python

*Алла Тамбовцева, НИУ ВШЭ*

Посмотрим на другие примеры использования `selenium`. 

**Пример.** Зайдем на сайт книжного магазина и найдем все книги В.Пелевина. Загрузим библиотеку, веб-драйвер и откроем страницу в браузере через Python.

In [11]:
from selenium import webdriver as wb
br = wb.Chrome()

# открываем страницу в Chrome в автоматическом режиме
br.get("http://www.biblio-globus.ru/")

Найдем с помощью CSS Selector'а поле для ввода названия книги или автора. 

In [12]:
field = br.find_element_by_css_selector("input")

Сохраним фамилию автора:

In [13]:
author = "Пелевин"

Введем фамилию автора в поле для поиска (`.send_keys`) и подождем чуть-чуть:

In [14]:
field.send_keys("Пелевин")
br.implicitly_wait(2) # подождем пару секунд

Теперь найдем кнопку для поиска (значок *лупа* рядом со строкой поиска) через CSS Selector:

In [17]:
submit = br.find_element_by_css_selector("#search_submit")

Кликнем на нее:

In [18]:
submit.click()

Сохраним первую страницу с результатами в переменную `page1`.

In [22]:
page1 = br.page_source

In [ ]:
page1

Теперь обработаем эту страницу через `BeautifulSoup`:

In [29]:
from bs4 import BeautifulSoup

In [31]:
soup1 = BeautifulSoup(page1, 'lxml')

Найдем все названия книг на этой странице. По исходному коду можно увидеть, что они имеют тэг 'a' с атрибутом `class`, равным `name`:

In [33]:
soup1.find_all('a', {'class':'name'})

[<a class="name" href="/search/catalog/details/10234458">Чапаев и Пустота</a>,
 <a class="name" href="/search/catalog/details/10480299">Бэтман Аполло</a>,
 <a class="name" href="/search/catalog/details/10231526">Священная книга оборотня</a>,
 <a class="name" href="/search/catalog/details/10332074">Бубен Верхнего Мира</a>,
 <a class="name" href="/search/catalog/details/10474602">Чапаев и Пустота</a>,
 <a class="name" href="/search/catalog/details/10477404">S.N.U.F.F.</a>,
 <a class="name" href="/search/catalog/details/9548086">Empire "V"</a>,
 <a class="name" href="/search/catalog/details/10215096">Смотритель. Книга 1. Орден желтого флага</a>,
 <a class="name" href="/search/catalog/details/10222210">Смотритель. В 2 т. Т. 2 : Железная бездна</a>,
 <a class="name" href="/search/catalog/details/10423839">iPhuck 10</a>]

С помощью списковых включений выберем из ссылок с тэгом `<a>` текст (так мы уже делали, и не раз).

In [34]:
books1 = [b.text for b in soup1.find_all('a', {'class':'name'})]

In [35]:
books1

['Чапаев и Пустота',
 'Бэтман Аполло',
 'Священная книга оборотня',
 'Бубен Верхнего Мира',
 'Чапаев и Пустота',
 'S.N.U.F.F.',
 'Empire "V"',
 'Смотритель. Книга 1. Орден желтого флага',
 'Смотритель. В 2 т. Т. 2 : Железная бездна',
 'iPhuck 10']

Теперь аналогичным образом сгрузим информацию о наличии книг:

In [36]:
instore1 = [s.text for s in soup1.find_all('div', {'class':'title_data green'})]

In [37]:
instore1

['\n                        В наличии\n                    ',
 '\n                        В наличии\n                    ',
 '\n                        В наличии\n                    ',
 '\n                        В наличии\n                    ',
 '\n                        В наличии\n                    ',
 '\n                        В наличии\n                    ',
 '\n                        В наличии\n                    ',
 '\n                        В наличии\n                    ',
 '\n                        В наличии\n                    ',
 '\n                        В наличии\n                    ']

Уберем лишнее:

In [38]:
instore1 = [i.strip() for i in instore1] # убираем лишние пробелы

In [39]:
instore1

['В наличии',
 'В наличии',
 'В наличии',
 'В наличии',
 'В наличии',
 'В наличии',
 'В наличии',
 'В наличии',
 'В наличии',
 'В наличии']

Сгрузим расположение:

In [40]:
place1 = [p.text for p in soup1.find_all('div', {'class':'placement'})]

In [41]:
place1

['Расположение в торговом зале: Уровень 2, зал № 12, секция 07, шкаф 70, полка 05',
 'Расположение в торговом зале: Уровень 2, зал № 12, секция 07, шкаф 70, полка 05',
 'Расположение в торговом зале: Уровень 2, зал № 12, секция 07, шкаф 70, полка 05',
 'Расположение в торговом зале: Уровень 2, зал № 12, секция 07, шкаф 70, полка 05',
 'Расположение в торговом зале: Уровень 2, зал № 12, секция 07, шкаф 70, полка 04',
 'Расположение в торговом зале: Уровень 2, зал № 12, секция 05, шкаф 54, полка 07',
 'Расположение в торговом зале: Уровень 2, зал № 12, секция 07, шкаф 70, полка 05',
 'Расположение в торговом зале: Уровень 2, зал № 12, секция 07, шкаф 70, полка 04',
 'Расположение в торговом зале: Уровень 2, зал № 12, секция 07, шкаф 70, полка 04',
 'Расположение в торговом зале: Уровень 2, зал № 12, секция 07, шкаф 70, полка 05']

И, конечно, цену:

In [42]:
price1 = [p.text for p in soup1.find_all('div', {'class':'title_data price'})]

In [43]:
price1

['Цена: 339,00 руб.',
 'Цена: 279,00 руб.',
 'Цена: 229,00 руб.',
 'Цена: 259,00 руб.',
 'Цена: 639,00 руб.',
 'Цена: 299,00 руб.',
 'Цена: 299,00 руб.',
 'Цена: 549,00 руб.',
 'Цена: 549,00 руб.',
 'Цена: 709,00 руб.']

Осталось пройтись по всем страницам, которые были выданы в результате поиска. Для примера перейдем на страницу 2 и на этом остановимся.

In [44]:
next_p = br.find_element_by_css_selector('.next_page')

In [45]:
next_p.click()

Проделаем то же самое, что и с первой страницей. По-хорошему нужно написать функцию, которая будет искать на странице названия книг, их расположение и цену. Но оставим это в качестве задания читателю :)

In [46]:
page2 = br.page_source
soup2 = BeautifulSoup(page2, 'lxml')
books2 = [b.text for b in soup2.find_all('a', {'class':'name'})]
instore2 = [s.text for s in soup2.find_all('div', {'class':'title_data green'})]
instore2 = [i.strip() for i in instore2] 
place2 = [p.text for p in soup2.find_all('div', {'class':'placement'})]
price2 = [p.text for p in soup2.find_all('div', {'class':'title_data price'})]

Расширим списки результатов с первой страницы данными, полученными со второй страницы, используя метод `.extend()`.

In [47]:
books1.extend(books2)
instore1.extend(instore2)
place1.extend(place2)
price1.extend(price2)

Осталось импортировать библиотеку `pandas` и создать датафрейм.

In [48]:
import pandas as pd

Для разнообразия создадим датафрейм не из списка списков, а из словаря. Ключами словаря будут названия столбцов в таблице, а значениями ‒ списки с сохраненной информацией (названия книг, цены и проч.).

In [51]:
df = pd.DataFrame({'books': books1, 'in_store': instore1, 'placement': place1, 'price': price1})

In [52]:
df.head()

,books,in_store,placement,price
0,Чапаев и Пустота,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 339,00 руб."
1,Бэтман Аполло,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 279,00 руб."
2,Священная книга оборотня,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 229,00 руб."
3,Бубен Верхнего Мира,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 259,00 руб."
4,Чапаев и Пустота,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 639,00 руб."


Давайте приведем столбец с ценой к числовому типу. Уберем слова *Цена* и *руб*, а потом сконвертируем строки в числа с плавающей точкой. Напишем функцию `get_price()`,

In [53]:
def get_price(price):
    book_price = price.split(' ')[1] # разобьем строку по пробелу и возьмем второй элемент
    book_price = book_price.replace(',', '.') # заменим запятую на точку
    price_num = float(book_price) # сконвертируем в float
    return price_num

In [54]:
# проверка
get_price(df.price[0])

339.0

Всё отлично работает! Применим функцию к столбцу *price* и создадим новый столбец *nprice*.

In [55]:
df['nprice'] = df.price.apply(get_price)

In [56]:
df.head()

,books,in_store,placement,price,nprice
0,Чапаев и Пустота,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 339,00 руб.",339.0
1,Бэтман Аполло,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 279,00 руб.",279.0
2,Священная книга оборотня,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 229,00 руб.",229.0
3,Бубен Верхнего Мира,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 259,00 руб.",259.0
4,Чапаев и Пустота,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 639,00 руб.",639.0


Теперь можем расположить книги по цене в порядке возрастания:

In [66]:
df.sort_values('nprice')

,books,in_store,placement,price,nprice
14,Жизнь насекомых,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 199,00 руб.",199.0
2,Священная книга оборотня,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 229,00 руб.",229.0
15,Empire V,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 229,00 руб.",229.0
3,Бубен Верхнего Мира,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 259,00 руб.",259.0
1,Бэтман Аполло,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 279,00 руб.",279.0
5,S.N.U.F.F.,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 299,00 руб.",299.0
6,"Empire ""V""",В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 299,00 руб.",299.0
0,Чапаев и Пустота,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 339,00 руб.",339.0
13,"Empire ""V""",В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 339,00 руб.",339.0
11,T,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 339,00 руб.",339.0


Выберем какую-нибудь книгу:

In [72]:
df[df.books == "Generation «П»"]

,books,in_store,placement,price,nprice
16,Generation «П»,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 709,00 руб.",709.0
17,Generation «П»,В наличии,"Расположение в торговом зале: Уровень 2, зал №...","Цена: 549,00 руб.",549.0


И сохраним всю таблицу в csv-файл:

In [73]:
df.to_csv("books.csv")